In [1]:
import tensorflow as tf
import numpy as np
import os

2024-08-17 20:03:05.840902: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-17 20:03:05.849382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 20:03:05.859121: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 20:03:05.862096: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-17 20:03:05.869321: I tensorflow/core/platform/cpu_feature_guar

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


I0000 00:00:1723905187.723272    8188 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723905187.727902    8188 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723905187.731531    8188 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723905187.735793    8188 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [4]:
file

'/home/subham/.keras/datasets/shakespeare.txt'

In [5]:
text = open(file,'rb').read().decode(encoding='utf-8')

In [6]:
type(text)

str

In [7]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [8]:
len(text)

1115394

In [9]:
len(set(text))

65

In [10]:
vocab = sorted(set(text))

In [11]:
c2i = {a:i for i,a in enumerate(vocab)}

In [12]:
i2c = np.array(vocab)

In [13]:
texts_as_sequence_of_int = np.array([c2i[a] for a in text])

In [14]:
texts_as_sequence_of_int[:10]

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

In [15]:
i2c[texts_as_sequence_of_int[:10]]

array(['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i'], dtype='<U1')

In [16]:
seq_length = 120
example_per_epochs = len(text)//(seq_length+1)

In [17]:
char_dataset = tf.data.Dataset.from_tensor_slices(texts_as_sequence_of_int)

In [18]:
for i in char_dataset.take(5):
    print(i2c[i.numpy()],i.numpy())

F 18
i 47
r 56
s 57
t 58


2024-08-17 20:03:15.388316: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [19]:
seq = char_dataset.batch(seq_length+1,drop_remainder=True)

In [20]:
def f_make_input_target_pairs(s):
    input_text = s[:-1]
    target_text = s[1:]
    return input_text,target_text

In [21]:
dataset = seq.map(f_make_input_target_pairs)

In [22]:
for X,y in dataset.take(1):
    print(i2c[X.numpy()])
    print(i2c[y.numpy()])

['F' 'i' 'r' 's' 't' ' ' 'C' 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'B' 'e' 'f'
 'o' 'r' 'e' ' ' 'w' 'e' ' ' 'p' 'r' 'o' 'c' 'e' 'e' 'd' ' ' 'a' 'n' 'y'
 ' ' 'f' 'u' 'r' 't' 'h' 'e' 'r' ',' ' ' 'h' 'e' 'a' 'r' ' ' 'm' 'e' ' '
 's' 'p' 'e' 'a' 'k' '.' '\n' '\n' 'A' 'l' 'l' ':' '\n' 'S' 'p' 'e' 'a'
 'k' ',' ' ' 's' 'p' 'e' 'a' 'k' '.' '\n' '\n' 'F' 'i' 'r' 's' 't' ' ' 'C'
 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'Y' 'o' 'u' ' ' 'a' 'r' 'e' ' ' 'a' 'l'
 'l' ' ' 'r' 'e' 's' 'o' 'l' 'v' 'e' 'd' ' ' 'r' 'a']
['i' 'r' 's' 't' ' ' 'C' 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'B' 'e' 'f' 'o'
 'r' 'e' ' ' 'w' 'e' ' ' 'p' 'r' 'o' 'c' 'e' 'e' 'd' ' ' 'a' 'n' 'y' ' '
 'f' 'u' 'r' 't' 'h' 'e' 'r' ',' ' ' 'h' 'e' 'a' 'r' ' ' 'm' 'e' ' ' 's'
 'p' 'e' 'a' 'k' '.' '\n' '\n' 'A' 'l' 'l' ':' '\n' 'S' 'p' 'e' 'a' 'k'
 ',' ' ' 's' 'p' 'e' 'a' 'k' '.' '\n' '\n' 'F' 'i' 'r' 's' 't' ' ' 'C' 'i'
 't' 'i' 'z' 'e' 'n' ':' '\n' 'Y' 'o' 'u' ' ' 'a' 'r' 'e' ' ' 'a' 'l' 'l'
 ' ' 'r' 'e' 's' 'o' 'l' 'v' 'e' 'd' ' ' 'r' 'a' 't']


2024-08-17 20:03:16.908149: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [23]:
BS = 50
dataset = dataset.shuffle(1000).batch(BS,drop_remainder=True)

In [24]:
VS = len(vocab)
ED = 100
ND = 1024

In [25]:
def f_make_model(VS,ED,ND,BS):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VS,ED),
        tf.keras.layers.GRU(ND,return_sequences=True,
                           stateful=True),
        tf.keras.layers.Dense(VS)
    ])
    return model

In [26]:
model = f_make_model(VS,ED,ND,BS)

In [27]:
def f_loss(y,y_hat):
    return tf.keras.losses.sparse_categorical_crossentropy(y,y_hat,from_logits=True)

In [28]:
model.compile(optimizer='adam',loss=f_loss)

In [29]:
checkpointDir = './tr_checkpoint'
batch = BS
checkpoint_prefix = os.path.join(checkpointDir,'chpt_50.weights.h5')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [30]:
history = model.fit(dataset,epochs=15,callbacks=[checkpoint_callback])

Epoch 1/15
  1/184 ━━━━━━━━━━━━━━━━━━━━ 4:57 2s/step - loss: 4.1743

2024-08-17 20:03:22.551593: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


184/184 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - loss: 3.1843
Epoch 2/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 2.0398
Epoch 3/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.7600
Epoch 4/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.5872
Epoch 5/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.4796
Epoch 6/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.4091
Epoch 7/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.3594
Epoch 8/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.3171
Epoch 9/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.2786
Epoch 10/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.2455
Epoch 11/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.2136
Epoch 12/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.1803
Epoch 13/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.1490
Epoch 14/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.1167
Epoch 15/15
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step 

In [31]:
model_1 = f_make_model(VS,ED,ND,1)

In [36]:
model_1.load_weights(tf.train.load_checkpoint(checkpoint_callback))

TypeError: Expected binary or unicode string, got <keras.src.callbacks.model_checkpoint.ModelCheckpoint object at 0x7ed89e512b80>

In [37]:
model.build(tf.TensorShape([1,None]))

In [42]:
def f_write_now(model,ss):
    N = 10000
    ie = [c2i[a] for a in ss]
    ie = tf.expand_dims(ie,axis=0)
    g_text = []
    # model.reset_states(stateful=True)
    for i in range(N):
        p = model(ie)
        print(p.shape)
        p = np.squeeze(p,axis=0)
        p_id = tf.random.categorical(p,num_samples=1)[-1,0].numpy()
        ie = tf.expand_dims(p_id,axis=0)
        g_text.append(i2c[p_id])
    retutn (ss + ''.join(g_text))

In [43]:
print(f_write_now(model,ss=u"ROMEO: "))

(50, 7, 65)


2024-08-17 20:09:26.595465: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at cudnn_rnn_ops.cc:1723 : INVALID_ARGUMENT: Invalid input_h shape: [50,1,1024] [1,1,1024]


ValueError: cannot select an axis to squeeze out which has size not equal to one

In [128]:
print(VS)

65
